In [1]:
import csv
import requests
import bs4
import io

# Extraction

In [2]:
def extract(url):
    r = requests.get(url) #returns the HTML of the page, can be done through urlopen as well
    soup = bs4.BeautifulSoup(r.content)
    
    titles = soup.findAll("td",{"style":"border:0 none;padding:0;background:0 none;"})
    titles = titles[1:len(titles):2]
    
    majors = soup.findAll("td",{"width":"135"})
    
    languages = soup.findAll("td",{"width":"105"})
    
    start = soup.findAll("td",{"width":"120"})
    start = start[0:len(start):2]
    
    scholarship = soup.findAll("td",{"width":"212"})
    
    to_pay = soup.findAll("td",{"width":"262"})
    
    return titles, majors, languages, start, scholarship, to_pay

In [3]:
def page_count(url):
    url = url + str(1)
    r = requests.get(url) #returns the HTML of the page, can be done through urlopen as well
    soup = bs4.BeautifulSoup(r.content)
    page_numbers = soup.findAll("div",{"class":"page"})
    total_schols = int(page_numbers[0].text[page_numbers[0].text.find(':')+1:])
    print('ttl page ' + str(total_schols))
    if (total_schols%15==0):
        iterations = total_schols/15
    else:
        iterations = int(total_schols/15)+1
    return(int(iterations))

In [4]:
dict = {}
schol_id=0
with open('test_new.csv', 'w', newline='',encoding="utf-8") as fp:
    a = csv.writer(fp, delimiter=',')
    a.writerows([['schol_id','University','Major','Level','Language','Start Date','Tuition Covered', 'Accomodation covered?', 'Living Expense Covered?','Tuition fees to pay','Original Tuition fee','Accomodation to pay','Living expense to pay']])
    for major_count in range(1,5):
        url = 'https://www.cucas.edu.cn/china_scholarships?cat=&degree=' + str(major_count) + '&school=&program=&lang=&semester=&page='
        iterations = page_count(url)
        
        for j in range(1,iterations+1):
            url_new = url + str(j)
            titles, majors , languages, start, scholarship, to_pay = extract(url_new)
            for i in range(0,len(majors)):
                title = titles[i].p.text
                major = majors[i].p.text
                if major_count==1:
                    level = 'Non-Degree'
                elif major_count==2:
                    level = 'Bachelor'
                elif major_count==3:
                    level = 'Master'
                elif major_count==4:
                    level = 'Phd'
                language = languages[i].text
                start_date = start[i].text

                tuition_acc_living = scholarship[i].p.text
                tuition = tuition_acc_living[tuition_acc_living.find(':')+1:tuition_acc_living.find('\n')].strip()

                accomodation = tuition_acc_living[tuition_acc_living.find(':',tuition_acc_living.find('Accom'))+1:tuition_acc_living.find('\n',tuition_acc_living.find('Accom'))].strip()

                living = tuition_acc_living[tuition_acc_living.find(':',tuition_acc_living.find('Living'))+1:].strip()

                To_Pay = to_pay[i].p.text
                try:
                    orig_tuition = to_pay[i].p.span.text.strip()
                except:
                    orig_tuition=-1
                try:
                    tuition_to_pay = To_Pay[To_Pay.find(':')+1:To_Pay.find('\n',To_Pay.find(':')+1)].strip().replace(orig_tuition,'').strip()
                except:
                    tuition_to_pay = To_Pay[To_Pay.find(':')+1:To_Pay.find('\n', To_Pay.find(':'))].strip()
                
                try:
                    accomodation_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Acco'))+1:To_Pay.find('\n',To_Pay.find(':',To_Pay.find('Acco')))].strip()
                except:
                    accomodation_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Acc'))+1:To_Pay.find('\n',To_Pay.find('Acc'))].strip()
                try:
                    Living_Expense_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Expense'))+1:To_Pay.find('\n',To_Pay.find(':',To_Pay.find('Expense')))].strip()
                except:
                    Living_Expense_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Liv'))+1:To_Pay.find('\n',To_Pay.find('Liv'))].strip()
                
                schol_id+=1
                data = [[schol_id,title, major,level, language, start_date,tuition,accomodation, living, tuition_to_pay, orig_tuition,accomodation_to_pay,Living_Expense_to_pay]]
                a.writerows(data)
print('done')

ttl page 62
ttl page 1390
ttl page 1461
ttl page 692
done
